In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

In [2]:
data = pd.read_csv('서울특별시 공공자전거 대여정보_201809_2.csv', encoding='utf-8')

C:\Users\kwanyong\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (0,1,3,5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data.columns

Index(['자전거번호', '대여일시', '대여소번호', '대여대여소명', '대여거치대', '반납일시', '반납대여소번호',
       '반납대여소명', '반납거치대', '이용시간', '이용거리'],
      dtype='object')

In [4]:
data.describe()

,대여소번호,대여거치대,반납대여소번호,반납거치대,이용시간,이용거리
count,385067.000000,385067.000000,385067.000000,385067.000000,385067.000000,385067.000000
mean,1116.927002,7.167670,1114.703018,7.166394,35.783778,5559.648191
std,807.286235,5.421831,804.829957,5.466760,34.501374,11086.654382
min,3.000000,1.000000,3.000000,1.000000,2.000000,0.000000
25%,402.000000,3.000000,407.000000,3.000000,10.000000,1480.000000
50%,1021.000000,6.000000,1020.000000,6.000000,24.000000,3020.000000
75%,1667.000000,10.000000,1663.000000,10.000000,51.000000,6450.000000
max,3524.000000,40.000000,3524.000000,40.000000,1118.000000,255550.000000


In [6]:
data.head(10)

,자전거번호,대여일시,대여소번호,대여대여소명,대여거치대,반납일시,반납대여소번호,반납대여소명,반납거치대,이용시간,이용거리
0,SPB-11299,2018-09-23 20:46,125.0,서강대 남문 옆,2.0,2018-09-23 20:50,129.0,신촌역(2호선) 6번출구 옆,9.0,4.0,670.0
1,SPB-14425,2018-09-23 20:46,1518.0,SK telecom(수유직영점) 앞,9.0,2018-09-23 20:50,1512.0,강북중학교 앞,19.0,3.0,1150.0
2,SPB-05366,2018-09-23 20:47,3501.0,광진구청 앞,3.0,2018-09-23 20:50,3520.0,광진경찰서,5.0,2.0,360.0
3,SPB-05791,2018-09-23 20:45,2307.0,압구정 한양 3차 아파트,1.0,2018-09-23 20:50,2307.0,압구정 한양 3차 아파트,1.0,4.0,360.0
4,SPB-18318,2018-09-23 20:47,591.0,건국대학교 (행정관),1.0,2018-09-23 20:50,503.0,더샵스타시티 C동 앞,15.0,3.0,850.0
5,SPB-14784,2018-09-23 20:46,1158.0,가양역 8번출구,17.0,2018-09-23 20:50,1118.0,증미역 3번출구뒤(등촌두산위브센티움오피스텔),3.0,3.0,750.0
6,SPB-08430,2018-09-23 20:48,1628.0,노일초등학교 앞,6.0,2018-09-23 20:50,1627.0,수락산역 4번출구,10.0,2.0,450.0
7,SPB-06274,2018-09-23 20:45,1974.0,신도림2차e편한세상아파트 203동 앞,7.0,2018-09-23 20:50,1901.0,신도림동주민센터 앞,6.0,5.0,0.0
8,SPB-05881,2018-09-23 20:45,1636.0,백병원 사거리 농협은행 앞,7.0,2018-09-23 20:50,1661.0,당현천근린공원,7.0,5.0,410.0
9,SPB-07707,2018-09-23 20:45,407.0,마포구 육아종합지원센터,2.0,2018-09-23 20:50,414.0,상암동주민센터 옆,5.0,5.0,1080.0


In [44]:
merge_data = pd.read_csv('공공자전거 대여소 정보_201905.csv', encoding='utf-8')

In [45]:
data = data.merge(merge_data, left_on='대여소번호', right_on='대여소번호')

In [46]:
data.head()

,자전거번호,대여일시,대여소번호,대여대여소명,대여거치대,반납일시,반납대여소번호,반납대여소명,반납거치대,이용시간,이용거리,구분,대여소명,위도,경도,거치대수
0,SPB-11299,2018-09-23 20:46,125.0,서강대 남문 옆,2.0,2018-09-23 20:50,129.0,신촌역(2호선) 6번출구 옆,9.0,4.0,670.0,마포구,125. 서강대 남문 옆,37.549484,126.93895,15
1,SPB-07124,2018-09-23 21:56,125.0,서강대 남문 옆,3.0,2018-09-23 22:01,129.0,신촌역(2호선) 6번출구 옆,12.0,4.0,660.0,마포구,125. 서강대 남문 옆,37.549484,126.93895,15
2,SPB-06587,2018-09-23 22:38,125.0,서강대 남문 옆,3.0,2018-09-23 22:45,128.0,신촌역(2호선) 1번출구 옆,11.0,6.0,760.0,마포구,125. 서강대 남문 옆,37.549484,126.93895,15
3,SPB-05105,2018-09-23 22:57,125.0,서강대 남문 옆,6.0,2018-09-23 23:38,374.0,청구역 2번출구 앞,10.0,41.0,10560.0,마포구,125. 서강대 남문 옆,37.549484,126.93895,15
4,SPB-15555,2018-09-24 0:57,125.0,서강대 남문 옆,12.0,2018-09-24 1:12,426.0,서울신용보증재단,8.0,14.0,1900.0,마포구,125. 서강대 남문 옆,37.549484,126.93895,15


In [47]:
data = data.drop(columns=['자전거번호','대여거치대','반납거치대','대여소명','거치대수'])

In [48]:
data.head()

,대여일시,대여소번호,대여대여소명,반납일시,반납대여소번호,반납대여소명,이용시간,이용거리,구분,위도,경도
0,2018-09-23 20:46,125.0,서강대 남문 옆,2018-09-23 20:50,129.0,신촌역(2호선) 6번출구 옆,4.0,670.0,마포구,37.549484,126.93895
1,2018-09-23 21:56,125.0,서강대 남문 옆,2018-09-23 22:01,129.0,신촌역(2호선) 6번출구 옆,4.0,660.0,마포구,37.549484,126.93895
2,2018-09-23 22:38,125.0,서강대 남문 옆,2018-09-23 22:45,128.0,신촌역(2호선) 1번출구 옆,6.0,760.0,마포구,37.549484,126.93895
3,2018-09-23 22:57,125.0,서강대 남문 옆,2018-09-23 23:38,374.0,청구역 2번출구 앞,41.0,10560.0,마포구,37.549484,126.93895
4,2018-09-24 0:57,125.0,서강대 남문 옆,2018-09-24 1:12,426.0,서울신용보증재단,14.0,1900.0,마포구,37.549484,126.93895


In [50]:
data = data.rename(columns={'구분':'대여소 자치구', '위도':'대여소 위도', '경도':'대여소 경도'})

In [51]:
data = data.merge(merge_data, left_on='반납대여소번호', right_on='대여소번호')

In [52]:
data.head()

,대여일시,대여소번호_x,대여대여소명,반납일시,반납대여소번호,반납대여소명,이용시간,이용거리,대여소 자치구,대여소 위도,대여소 경도,구분,대여소번호_y,대여소명,위도,경도,거치대수
0,2018-09-23 20:46,125.0,서강대 남문 옆,2018-09-23 20:50,129.0,신촌역(2호선) 6번출구 옆,4.0,670.0,마포구,37.549484,126.93895,마포구,129,129. 신촌역(2호선) 6번출구 옆,37.555054,126.937569,15
1,2018-09-23 21:56,125.0,서강대 남문 옆,2018-09-23 22:01,129.0,신촌역(2호선) 6번출구 옆,4.0,660.0,마포구,37.549484,126.93895,마포구,129,129. 신촌역(2호선) 6번출구 옆,37.555054,126.937569,15
2,2018-09-24 11:44,125.0,서강대 남문 옆,2018-09-24 11:48,129.0,신촌역(2호선) 6번출구 옆,3.0,650.0,마포구,37.549484,126.93895,마포구,129,129. 신촌역(2호선) 6번출구 옆,37.555054,126.937569,15
3,2018-09-24 12:48,125.0,서강대 남문 옆,2018-09-24 12:55,129.0,신촌역(2호선) 6번출구 옆,5.0,1010.0,마포구,37.549484,126.93895,마포구,129,129. 신촌역(2호선) 6번출구 옆,37.555054,126.937569,15
4,2018-09-25 16:41,125.0,서강대 남문 옆,2018-09-25 16:45,129.0,신촌역(2호선) 6번출구 옆,3.0,660.0,마포구,37.549484,126.93895,마포구,129,129. 신촌역(2호선) 6번출구 옆,37.555054,126.937569,15


In [53]:
data = data.drop(columns=['대여소번호_y','거치대수'])
data = data.rename(columns={'대여소번호_x':'대여소번호','구분':'반납 대여소 자치구', '위도':'반납 대여소 위도', '경도':'반납 대여소 경도'})

In [55]:
data = data.drop(columns=['대여소명'])

In [56]:
data.head()

,대여일시,대여소번호,대여대여소명,반납일시,반납대여소번호,반납대여소명,이용시간,이용거리,대여소 자치구,대여소 위도,대여소 경도,반납 대여소 자치구,반납 대여소 위도,반납 대여소 경도
0,2018-09-23 20:46,125.0,서강대 남문 옆,2018-09-23 20:50,129.0,신촌역(2호선) 6번출구 옆,4.0,670.0,마포구,37.549484,126.93895,마포구,37.555054,126.937569
1,2018-09-23 21:56,125.0,서강대 남문 옆,2018-09-23 22:01,129.0,신촌역(2호선) 6번출구 옆,4.0,660.0,마포구,37.549484,126.93895,마포구,37.555054,126.937569
2,2018-09-24 11:44,125.0,서강대 남문 옆,2018-09-24 11:48,129.0,신촌역(2호선) 6번출구 옆,3.0,650.0,마포구,37.549484,126.93895,마포구,37.555054,126.937569
3,2018-09-24 12:48,125.0,서강대 남문 옆,2018-09-24 12:55,129.0,신촌역(2호선) 6번출구 옆,5.0,1010.0,마포구,37.549484,126.93895,마포구,37.555054,126.937569
4,2018-09-25 16:41,125.0,서강대 남문 옆,2018-09-25 16:45,129.0,신촌역(2호선) 6번출구 옆,3.0,660.0,마포구,37.549484,126.93895,마포구,37.555054,126.937569


In [58]:
data.to_csv('따릉이_sample.csv')

In [ ]:
for i in range(1,13):
    if i<10:
        data = pd.read_csv('공공자전거 대여소 정보_20190'+i+'.csv', encoding='utf-8')
    else:
        data = pd.read_csv('공공자전거 대여소 정보_2019'+i+'.csv', encoding='utf-8')
    

In [64]:
merge_data = pd.read_csv('공공자전거 대여소 정보_201905.csv', encoding='utf-8')
merge_data.drop(columns=['대여소명','거치대수'])

,구분,대여소번호,위도,경도
0,마포구,101,37.549561,126.905754
1,마포구,102,37.556000,126.910454
2,마포구,103,37.554951,126.910835
3,마포구,104,37.550629,126.914986
4,마포구,105,37.550007,126.914825
5,마포구,106,37.548645,126.912827
6,마포구,107,37.557510,126.918503
7,마포구,108,37.552746,126.918617
8,마포구,109,37.547691,126.919983
9,서대문구,110,37.568199,126.917847


In [69]:
def create_data(csv):
    data = pd.read_csv(csv+'.csv', encoding='utf-8')
    data = data.merge(merge_data, left_on='대여소번호', right_on='대여소번호')
    data = data.drop(columns=['자전거번호','대여거치대','반납거치대'])
    data = data.rename(columns={'구분':'대여소 자치구', '위도':'대여소 위도', '경도':'대여소 경도'})
    data = data.merge(merge_data, left_on='반납대여소번호', right_on='대여소번호')
    data = data.drop(columns=['대여소번호_y','대여소명_x','거치대수_x','대여소명_y','거치대수_y'])
    data = data.rename(columns={'대여소번호_x':'대여소번호','구분':'반납 대여소 자치구', '위도':'반납 대여소 위도', '경도':'반납 대여소 경도'})
    return data

In [70]:
a = '서울특별시 공공자전거 대여정보_201809_2'
data = create_data(a)

In [71]:
data.head()

,대여일시,대여소번호,대여대여소명,반납일시,반납대여소번호,반납대여소명,이용시간,이용거리,대여소 자치구,대여소 위도,대여소 경도,반납 대여소 자치구,반납 대여소 위도,반납 대여소 경도
0,2018-09-23 20:46,125.0,서강대 남문 옆,2018-09-23 20:50,129.0,신촌역(2호선) 6번출구 옆,4.0,670.0,마포구,37.549484,126.93895,마포구,37.555054,126.937569
1,2018-09-23 21:56,125.0,서강대 남문 옆,2018-09-23 22:01,129.0,신촌역(2호선) 6번출구 옆,4.0,660.0,마포구,37.549484,126.93895,마포구,37.555054,126.937569
2,2018-09-24 11:44,125.0,서강대 남문 옆,2018-09-24 11:48,129.0,신촌역(2호선) 6번출구 옆,3.0,650.0,마포구,37.549484,126.93895,마포구,37.555054,126.937569
3,2018-09-24 12:48,125.0,서강대 남문 옆,2018-09-24 12:55,129.0,신촌역(2호선) 6번출구 옆,5.0,1010.0,마포구,37.549484,126.93895,마포구,37.555054,126.937569
4,2018-09-25 16:41,125.0,서강대 남문 옆,2018-09-25 16:45,129.0,신촌역(2호선) 6번출구 옆,3.0,660.0,마포구,37.549484,126.93895,마포구,37.555054,126.937569


In [76]:
print(data['대여소 위도'].unique().mean(), data['대여소 경도'].unique().mean())

37.54700654279106 126.99246384892636


In [72]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import HeatMap

In [77]:
# define a function to draw a basemap easily
def generateBaseMap(default_location=[37.547, 126.992], default_zoom_start=9.4):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map

data_copy = data.copy()
# select a zipcode for the heatmap
# set(home['zipcode'])
# home_copy = home[home['zipcode']==98178].copy()
# data_copy['count'] = 1
basemap = generateBaseMap()
# add carton position map
folium.TileLayer('cartodbpositron').add_to(basemap)
s=folium.FeatureGroup(name='icon').add_to(basemap)
# add a marker for the house which has the highest price
# folium.Marker([maxpr['lat'], maxpr['long']],popup='Highest Price: $'+str(format(maxpr['price'],'.0f')),
#               icon=folium.Icon(color='red')).add_to(s)
# add heatmap
HeatMap(data=data_copy[['대여소 위도','대여소 경도','이용시간']].groupby(['대여소 위도','대여소 경도']).mean().reset_index().values.tolist(),
        radius=8,max_zoom=13,name='Heat Map').add_to(basemap)
folium.LayerControl(collapsed=False).add_to(basemap)
basemap

In [78]:
# define a function to draw a basemap easily
def generateBaseMap(default_location=[37.547, 126.992], default_zoom_start=9.4):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map

data_copy = data.copy()
# select a zipcode for the heatmap
# set(home['zipcode'])
# home_copy = home[home['zipcode']==98178].copy()
# data_copy['count'] = 1
basemap = generateBaseMap()
# add carton position map
folium.TileLayer('cartodbpositron').add_to(basemap)
s=folium.FeatureGroup(name='icon').add_to(basemap)
# add a marker for the house which has the highest price
# folium.Marker([maxpr['lat'], maxpr['long']],popup='Highest Price: $'+str(format(maxpr['price'],'.0f')),
#               icon=folium.Icon(color='red')).add_to(s)
# add heatmap
HeatMap(data=data_copy[['반납 대여소 위도','반납 대여소 경도','이용시간']].groupby(['반납 대여소 위도','반납 대여소 경도']).mean().reset_index().values.tolist(),
        radius=8,max_zoom=13,name='Heat Map').add_to(basemap)
folium.LayerControl(collapsed=False).add_to(basemap)
basemap

In [81]:
data.head()

,대여일시,대여소번호,대여대여소명,반납일시,반납대여소번호,반납대여소명,이용시간,이용거리,대여소 자치구,대여소 위도,대여소 경도,반납 대여소 자치구,반납 대여소 위도,반납 대여소 경도
0,2018-09-23 20:46,125.0,서강대 남문 옆,2018-09-23 20:50,129.0,신촌역(2호선) 6번출구 옆,4.0,670.0,마포구,37.549484,126.93895,마포구,37.555054,126.937569
1,2018-09-23 21:56,125.0,서강대 남문 옆,2018-09-23 22:01,129.0,신촌역(2호선) 6번출구 옆,4.0,660.0,마포구,37.549484,126.93895,마포구,37.555054,126.937569
2,2018-09-24 11:44,125.0,서강대 남문 옆,2018-09-24 11:48,129.0,신촌역(2호선) 6번출구 옆,3.0,650.0,마포구,37.549484,126.93895,마포구,37.555054,126.937569
3,2018-09-24 12:48,125.0,서강대 남문 옆,2018-09-24 12:55,129.0,신촌역(2호선) 6번출구 옆,5.0,1010.0,마포구,37.549484,126.93895,마포구,37.555054,126.937569
4,2018-09-25 16:41,125.0,서강대 남문 옆,2018-09-25 16:45,129.0,신촌역(2호선) 6번출구 옆,3.0,660.0,마포구,37.549484,126.93895,마포구,37.555054,126.937569


In [82]:
data_point_groupby = data.groupby('대여대여소명').mean()

In [85]:
data_point_groupby.sort_values(by=['이용시간'],ascending=False)

,대여소번호,반납대여소번호,이용시간,이용거리,대여소 위도,대여소 경도,반납 대여소 위도,반납 대여소 경도
대여대여소명,,,,,,,,
개화동상사마을종점 버스정류장,1101.0,991.413043,72.326087,15428.478261,37.581612,126.798599,37.566073,126.835812
동작역 5번출구 동작주차공원,2037.0,1551.068592,67.223827,8932.021661,37.503189,126.977150,37.513746,126.980316
서울숲 남문 버스정류소 옆,510.0,1320.801282,63.465812,7193.824786,37.541222,127.043800,37.542080,127.052191
KT반포지사,2509.0,1953.718750,62.968750,7582.187500,37.498371,126.992462,37.506365,126.986336
신반포역 2번출구 앞,2214.0,1892.146605,62.777778,7576.666667,37.504116,126.997490,37.508738,126.989281
여의나루역 1번출구 앞,207.0,572.134200,62.144014,7825.664868,37.526989,126.932098,37.527334,126.932702
독산주공 14단지 버스정류소,1847.0,1494.526316,61.754386,9694.385965,37.461033,126.887100,37.487183,126.881709
서울숲 관리사무소,3515.0,2238.259489,60.848171,6713.947550,37.542816,127.042084,37.542909,127.045996
동작역 7번출구,2038.0,1549.155556,60.733333,6856.666667,37.503181,126.976089,37.512729,126.970128


In [94]:
# 1318개 대여소
len(data_point_groupby['대여소번호'].unique())

1318

In [105]:
print('60분 이상 이용 대여대여소 :\t', len(data_point_groupby[data_point_groupby['이용시간']>=60]))
print('50분 이상 이용 대여대여소 :\t', len(data_point_groupby[data_point_groupby['이용시간']>=50]))
print('40분 이상 이용 대여대여소 :\t', len(data_point_groupby[data_point_groupby['이용시간']>=40]))
print('30분 이상 이용 대여대여소 :\t', len(data_point_groupby[data_point_groupby['이용시간']>=30]))
print('20분 이상 이용 대여대여소 :\t', len(data_point_groupby[data_point_groupby['이용시간']>=20]))
print('10분 이상 이용 대여대여소 :\t', len(data_point_groupby[data_point_groupby['이용시간']>=10]))

60분 이상 이용 대여대여소 :	 11
50분 이상 이용 대여대여소 :	 80
40분 이상 이용 대여대여소 :	 246
30분 이상 이용 대여대여소 :	 782
20분 이상 이용 대여대여소 :	 1289
10분 이상 이용 대여대여소 :	 1318


In [115]:
data_over20_under30 = data[data['이용시간'].apply(lambda x: x>=20 and x<30)]
data_over30_under40 = data[data['이용시간'].apply(lambda x: x>=30 and x<40)]
data_over50_under60 = data[data['이용시간'].apply(lambda x: x>=50 and x<60)]
data_over60 = data[data['이용시간'].apply(lambda x: x>=60)]

In [112]:
data_over20_under30.describe()

,대여소번호,반납대여소번호,이용시간,이용거리,대여소 위도,대여소 경도,반납 대여소 위도,반납 대여소 경도
count,48381.000000,48381.000000,48381.000000,48381.000000,48381.000000,48381.000000,48381.000000,48381.000000
mean,1110.755958,1116.156280,24.165933,4404.097476,37.548519,126.996540,37.548588,126.996734
std,780.628169,782.089463,2.867048,10002.265140,0.045568,0.084202,0.045825,0.084659
min,101.000000,101.000000,20.000000,0.000000,37.443340,126.798599,37.443340,126.798599
25%,409.000000,418.000000,22.000000,2600.000000,37.513962,126.922447,37.513523,126.920837
50%,1035.000000,1042.000000,24.000000,3440.000000,37.545231,127.005959,37.545418,127.006729
75%,1653.000000,1651.000000,27.000000,4400.000000,37.573643,127.066223,37.574471,127.067192
max,3524.000000,3524.000000,29.000000,253600.000000,37.689720,127.180267,37.689720,127.180267


In [113]:
data_over30_under40.describe()

,대여소번호,반납대여소번호,이용시간,이용거리,대여소 위도,대여소 경도,반납 대여소 위도,반납 대여소 경도
count,34979.000000,34979.000000,34979.000000,34979.000000,34979.000000,34979.000000,34979.000000,34979.000000
mean,1106.304840,1120.938220,34.329798,5842.089539,37.546005,126.996205,37.546145,126.996480
std,794.366327,788.530929,2.888366,10553.055129,0.044298,0.083457,0.044967,0.084481
min,101.000000,101.000000,30.000000,0.000000,37.443340,126.798599,37.443340,126.798599
25%,401.000000,419.000000,32.000000,3460.000000,37.513523,126.922447,37.513248,126.921333
50%,1020.000000,1044.000000,34.000000,4750.000000,37.541718,127.005836,37.541412,127.006310
75%,1661.000000,1663.000000,37.000000,6220.000000,37.570480,127.066292,37.571140,127.067192
max,3524.000000,3524.000000,39.000000,253310.000000,37.689720,127.180267,37.689720,127.180267


In [114]:
data_over20_under30.to_csv('따릉이_20이상30미만.csv')
data_over30_under40.to_csv('따릉이_30이상40미만.csv')

In [116]:
data_over50_under60.to_csv('따릉이_50이상60미만.csv')
data_over60.to_csv('따릉이_60이상.csv')

In [117]:
data.describe()

,대여소번호,반납대여소번호,이용시간,이용거리,대여소 위도,대여소 경도,반납 대여소 위도,반납 대여소 경도
count,384967.000000,384967.000000,384967.000000,384967.000000,384967.000000,384967.000000,384967.000000,384967.000000
mean,1116.840527,1114.535181,35.781158,5559.387792,37.546522,126.994140,37.546487,126.994096
std,807.239876,804.718886,34.499029,11087.569589,0.044611,0.084139,0.044445,0.084541
min,101.000000,101.000000,2.000000,0.000000,37.443340,126.798599,37.443340,126.798599
25%,402.000000,407.000000,10.000000,1480.000000,37.513950,126.920837,37.513962,126.920128
50%,1021.000000,1020.000000,24.000000,3020.000000,37.542816,127.002457,37.542519,127.002533
75%,1667.000000,1663.000000,51.000000,6450.000000,37.571068,127.066200,37.570957,127.067192
max,3524.000000,3524.000000,1118.000000,255550.000000,37.689720,127.180267,37.689720,127.180267


In [118]:
data_distance_under75percent=data[data['이용거리']<=6450]

In [121]:
data_distance_25 = data_distance_under75percent[data_distance_under75percent['이용거리'].apply(lambda x: x<1480)]
data_distance_25_50 = data_distance_under75percent[data_distance_under75percent['이용거리'].apply(lambda x: x>=1480 and x<3020)]
data_distance_50_75 = data_distance_under75percent[data_distance_under75percent['이용거리'].apply(lambda x: x>=3020 and x<6450)]

In [122]:
data_distance_25.describe()

,대여소번호,반납대여소번호,이용시간,이용거리,대여소 위도,대여소 경도,반납 대여소 위도,반납 대여소 경도
count,96109.000000,96109.000000,96109.000000,96109.00000,96109.000000,96109.000000,96109.000000,96109.000000
mean,1091.278174,1086.040849,10.885776,871.63377,37.548662,126.993853,37.548688,126.993806
std,811.265402,805.847155,17.733468,400.05428,0.044494,0.084049,0.044465,0.084051
min,101.000000,101.000000,2.000000,0.00000,37.443340,126.798599,37.443340,126.798599
25%,374.000000,373.000000,4.000000,630.00000,37.516571,126.920837,37.516155,126.920837
50%,1001.000000,956.000000,6.000000,930.00000,37.547249,127.001785,37.547691,127.001785
75%,1637.000000,1637.000000,10.000000,1190.00000,37.573589,127.066292,37.573357,127.066200
max,3524.000000,3524.000000,933.000000,1470.00000,37.689720,127.180267,37.689720,127.180267


In [123]:
data_distance_25_50.describe()

,대여소번호,반납대여소번호,이용시간,이용거리,대여소 위도,대여소 경도,반납 대여소 위도,반납 대여소 경도
count,96011.000000,96011.000000,96011.000000,96011.000000,96011.000000,96011.000000,96011.000000,96011.000000
mean,1109.967993,1110.730718,20.252784,2164.778202,37.549761,126.996047,37.549728,126.995990
std,792.545095,790.963968,18.591471,441.469999,0.045704,0.085958,0.045718,0.086054
min,101.000000,101.000000,2.000000,1480.000000,37.443340,126.798599,37.443340,126.798599
25%,411.000000,411.000000,11.000000,1780.000000,37.515156,126.920837,37.515831,126.920128
50%,1026.000000,1027.000000,15.000000,2120.000000,37.548222,127.003944,37.548222,127.005539
75%,1627.000000,1627.000000,21.000000,2530.000000,37.574478,127.068329,37.574471,127.068329
max,3524.000000,3524.000000,1118.000000,3010.000000,37.689720,127.180267,37.689720,127.180267


In [124]:
data_distance_50_75.describe()

,대여소번호,반납대여소번호,이용시간,이용거리,대여소 위도,대여소 경도,반납 대여소 위도,반납 대여소 경도
count,96538.000000,96538.000000,96538.000000,96538.000000,96538.000000,96538.000000,96538.000000,96538.000000
mean,1111.607326,1112.159429,39.139479,4480.256686,37.545563,126.997103,37.545546,126.997291
std,801.059326,796.090278,23.675266,981.546865,0.044120,0.084248,0.044179,0.084806
min,101.000000,101.000000,2.000000,3020.000000,37.443340,126.798599,37.443340,126.798599
25%,402.000000,413.000000,24.000000,3620.000000,37.513577,126.923203,37.513523,126.923065
50%,1033.000000,1036.000000,33.000000,4360.000000,37.541222,127.005959,37.540730,127.006729
75%,1656.000000,1653.000000,47.000000,5280.000000,37.570198,127.067192,37.570396,127.067192
max,3524.000000,3524.000000,646.000000,6440.000000,37.689720,127.180267,37.689720,127.180267


In [126]:
data_distance_25.to_csv('거리기준_25퍼미만.csv')
data_distance_25_50.to_csv('거리기준_50퍼미만.csv')
data_distance_50_75.to_csv('거리기준_75퍼미만.csv')